In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/EnKF/")

"/media/mat/HDD/EnKF/Project.toml"

In [2]:
using Revise
using EnKF
using Distributions
using LinearAlgebra

┌ Info: Recompiling stale cache file /home/mat/.julia/compiled/v1.1/EnKF/oXK06.ji for EnKF [685896a8-a41b-11e9-3419-3315e75b5d74]
└ @ Base loading.jl:1184


In [3]:
f = PropagationFunction()
m = MeasurementFunction()
z = RealMeasurementFunction()

RealMeasurementFunction()

In [4]:
function (::PropagationFunction)(t::Float64, ENS::EnsembleState{N, NS, TS}) where {N, NS, TS}
    return ENS  
end

In [35]:
function (::MeasurementFunction)(t::Float64, mENS::EnsembleState{N, NZ, TZ},  ENS::EnsembleState{N, NS, TS}) where {N, NS, TS, NZ, TZ}
    fill!(mENS, 1.01*ones(NZ))
    return mENS
end

In [36]:
function (::MeasurementFunction)(t::Float64, S::Array{TS,1}) where {TS}
    return 1.05*ones(NZ)
end

In [37]:
function (::RealMeasurementFunction)(t::Float64, zENS::EnsembleState{N, NZ, TZ}) where {N, NZ, TZ} 
    fill!(zENS, 1.01*ones(NZ))
    return zENS
end

#### Ensemble Kalman filter

In [38]:
N = 10
NS = 3
TS = Float64
NZ = 5
TZ = Float64
β = MultiplicativeInflation(NS, 1.01)
ϵ = AdditiveInflation(NZ)
isinflated = true
isaugmented = false

false

In [39]:
enkf = ENKF{N, NS, TS, NZ, TZ}(f, β, m, z, ϵ, isinflated, isaugmented)

Ensemble Kalman filter with 10 members of state of length 3 and measurement vector of length 5

In [40]:
ens = EnsembleState(N, NS)

EnsembleState{10,3,Float64}(Array{Float64,1}[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

#### Initialize each ensemble member

In [41]:
initialize(ens)

EnsembleState{10,3,Float64}(Array{Float64,1}[[-0.226733, 0.595825, 0.852152], [-1.00955, -1.47032, 0.264316], [0.238853, 0.262367, 0.116789], [-0.98411, 0.774457, -0.289939], [-0.90929, 0.230529, 0.74508], [-1.57104, -0.737209, 1.12347], [-0.485561, -1.28475, -1.07686], [-1.59523, -0.804021, -1.26934], [-1.89688, 0.352831, 0.0642917], [1.29957, 1.41345, -1.08115]])

#### Propagate each ensemble member

In [42]:
enkf.f(0.0, ens)

EnsembleState{10,3,Float64}(Array{Float64,1}[[-0.226733, 0.595825, 0.852152], [-1.00955, -1.47032, 0.264316], [0.238853, 0.262367, 0.116789], [-0.98411, 0.774457, -0.289939], [-0.90929, 0.230529, 0.74508], [-1.57104, -0.737209, 1.12347], [-0.485561, -1.28475, -1.07686], [-1.59523, -0.804021, -1.26934], [-1.89688, 0.352831, 0.0642917], [1.29957, 1.41345, -1.08115]])

#### Apply convariance inflation

In [43]:
if isinflated ==true
        enkf.A(ens)
end

EnsembleState{10,3,Float64}(Array{Float64,1}[[-0.22186, 0.60245, 0.861225], [-1.0125, -1.48436, 0.267511], [0.248381, 0.265657, 0.118508], [-0.986811, 0.782868, -0.292288], [-0.911243, 0.233501, 0.753082], [-1.57962, -0.743914, 1.13525], [-0.483276, -1.29694, -1.08708], [-1.60404, -0.811394, -1.28148], [-1.90871, 0.357027, 0.0654858], [1.31971, 1.42825, -1.09141]])

In [44]:
ens

EnsembleState{10,3,Float64}(Array{Float64,1}[[-0.22186, 0.60245, 0.861225], [-1.0125, -1.48436, 0.267511], [0.248381, 0.265657, 0.118508], [-0.986811, 0.782868, -0.292288], [-0.911243, 0.233501, 0.753082], [-1.57962, -0.743914, 1.13525], [-0.483276, -1.29694, -1.08708], [-1.60404, -0.811394, -1.28148], [-1.90871, 0.357027, 0.0654858], [1.31971, 1.42825, -1.09141]])

#### Compute mean and deviation

In [45]:
Ŝ = deepcopy(mean(ens))

3-element Array{Float64,1}:
 -0.7139980004289856  
 -0.0666849922078431  
 -0.055119218357718346

In [66]:
ensfluc = EnsembleState(size(ens))

deviation(ensfluc, ens)
A′ = hcat(ensfluc)

3×10 Array{Float64,2}:
 0.492138  -0.298506  0.962379  -0.272813  …  -0.890046  -1.19472    2.0337 
 0.669135  -1.41767   0.332342   0.849553     -0.744709   0.423712   1.49493
 0.916344   0.32263   0.173627  -0.237168     -1.22636    0.120605  -1.03629

#### Compute measurement

In [90]:
mens = EnsembleState((N, NZ))
enkf.m(0.0, mens, ens)
Â = hcat(deepcopy(mens))

5×10 Array{Float64,2}:
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01
 1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01

#### Compute deviation from measurement of the mean

In [92]:
Â′  = Â .- enkf.m(0.0, mean(ens))

5×10 Array{Float64,2}:
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04
 -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04  -0.04

#### Get actual measurements

In [93]:
zens = EnsembleState((N, NZ))
M = 1.01*ones(NZ)
enkf.z(0.0, zens)

EnsembleState{10,5,Float64}(Array{Float64,1}[[1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01]])

In [94]:
zens

EnsembleState{10,5,Float64}(Array{Float64,1}[[1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01], [1.01, 1.01, 1.01, 1.01, 1.01]])

#### Perturb actual measurement

In [95]:
ϵ(zens)

EnsembleState{10,5,Float64}(Array{Float64,1}[[0.593424, 0.643529, 0.70924, 1.65074, 1.62482], [1.01748, 0.200967, 0.570789, 1.15389, 2.86036], [2.13993, -0.115244, 0.355753, -0.100707, 2.59735], [0.114062, 0.675312, -0.138637, 2.85334, 1.92207], [0.161205, 1.19694, 0.222819, 0.505954, 0.695853], [1.93208, 0.443653, 1.56064, 1.50817, 1.02439], [-0.837596, 2.89022, 2.03434, 0.211674, 3.22826], [-0.461287, 2.36064, 1.27148, 0.721252, -0.434497], [0.29815, 0.598726, -0.707126, 0.767909, 0.11082], [0.402255, -0.978067, -0.213465, 0.690756, 2.0126]])

#### Construct representer matrix and vectors

In [96]:
D = hcat(zens)

5×10 Array{Float64,2}:
 0.593424  1.01748    2.13993    0.114062  …  -0.461287   0.29815    0.402255
 0.643529  0.200967  -0.115244   0.675312      2.36064    0.598726  -0.978067
 0.70924   0.570789   0.355753  -0.138637      1.27148   -0.707126  -0.213465
 1.65074   1.15389   -0.100707   2.85334       0.721252   0.767909   0.690756
 1.62482   2.86036    2.59735    1.92207      -0.434497   0.11082    2.0126  

In [97]:
cov(ϵ)

5×5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0

In [98]:
(Â′*Â′') + (N-1)*cov(ϵ)*I

5×5 Array{Float64,2}:
 9.016  0.016  0.016  0.016  0.016
 0.016  9.016  0.016  0.016  0.016
 0.016  0.016  9.016  0.016  0.016
 0.016  0.016  0.016  9.016  0.016
 0.016  0.016  0.016  0.016  9.016

In [101]:
@time b = ((Â′*Â′') + (N-1)*cov(ϵ)*I) \ (D - Â)

  0.000160 seconds (18 allocations: 3.156 KiB)


5×10 Array{Float64,2}:
 -0.0463199   0.000683825   0.125581   …  -0.163165   -0.0783149  -0.0669133
 -0.0407526  -0.0900401    -0.124993       0.150383   -0.0449175  -0.220282 
 -0.0334514  -0.0489487    -0.0726603      0.0293653  -0.190012   -0.135327 
  0.0711599   0.0158403    -0.123378      -0.0317714  -0.0261194  -0.0348575
  0.06828     0.205448      0.176406      -0.160188   -0.0991294   0.112014 

In [102]:
@time Bᵀb = (A′*Â′')*b

  0.008230 seconds (4.05 k allocations: 225.549 KiB)


3×10 Array{Float64,2}:
  8.23539e-20   3.61285e-19  -8.29126e-20  …  -1.90906e-18  -1.50361e-18
 -4.32364e-19  -1.89677e-18   4.35298e-19      1.00227e-17   7.89408e-18
  4.65409e-19   2.04174e-18  -4.68566e-19     -1.07887e-17  -8.4974e-18 

#### Analysis step

In [107]:
ens  += cut(Bᵀb)

EnsembleState{10,3,Float64}(Array{Float64,1}[[-0.22186, 0.60245, 0.861225], [-1.0125, -1.48436, 0.267511], [0.248381, 0.265657, 0.118508], [-0.986811, 0.782868, -0.292288], [-0.911243, 0.233501, 0.753082], [-1.57962, -0.743914, 1.13525], [-0.483276, -1.29694, -1.08708], [-1.60404, -0.811394, -1.28148], [-1.90871, 0.357027, 0.0654858], [1.31971, 1.42825, -1.09141]])